In [1]:
import json
import pandas as pd

import portrisk

from deribit import get_order_book

pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [2]:
# Load stress parameters
file_path = './parameters/'

with open(file_path + 'spot_vol_shocks.json', 'r') as f:
    spot_vol_shocks_param = json.load(f)

crypto_parameters = portrisk.core.utils.CryptoParameters(spot_vol_shocks_param)

with open(file_path + 'vol_surface_shocks.json', 'r') as f:
    vol_surface_shocks_param = json.load(f)

vol_surface_shocks_parameters = portrisk.core.utils.VolSurfaceParameters(vol_surface_shocks_param)


In [ ]:
# load position data
df = pd.read_excel('positions.xlsx')

# Splitting the 'instrument' column by "-"
df[['underlying','expiry','strike','put_call']] = df['instrument'].str.split('-', expand=True)
df['strike'] = df['strike'].astype(float)
df['put_call'] = df['put_call'].map({'P':'put','C':'call'})
df['expiry'] = pd.to_datetime(df['expiry'], format='%d%b%y')

# Calculate time to expiry
valuation_day = pd.to_datetime('2024-08-05')
df['time_to_expiry'] = (df['expiry'] - valuation_day).dt.days / 365

# get market data from deribit
deribit_res = get_order_book(instruments=df['instrument'].to_list())
df = pd.merge(df, deribit_res,how='left',left_on='instrument',right_on='instrument_name')


# calculate input for BSM		
df['cost_of_carry_rate'] = 'default'
df['multiplier'] = 1
df['rate'] = 0.03
df['vol'] = df['mark_iv'] / 100
df['spot'] = df['index_price'].values[0]
df['position_vega'] = df['vega'] * df['quantity']

# calculate atm_ivol 

# atm_ivol_map = {}
# for u in df['underlying'].to_list():
#     tem = df[df['underlying'].isin([u])].copy()
#     # Find the closest value in column strike to the spot
#     closest_value = tem['strike'].iloc[(tem['strike'] - tem['spot'].values[0]).abs().argsort()[0]]
#     # Retrieve the corresponding value from column vol
#     atm_ivol_map[u] = tem.loc[df['strike'] == closest_value, 'vol'].values[0]

# df['atm_ivol'] = df['underlying'].map(atm_ivol_map)
# Function to get ATM implied volatility for a specific group

def get_atm_iv(group):
    group['diff'] = (group['strike'] - group['spot']).abs()
    atm_index = group['diff'].idxmin()
    atm_iv = group.at[atm_index, 'vol']
    group['atm_ivol'] = atm_iv
    return group

# Apply the function to each group
df = df.groupby('expiry',as_index=False, group_keys=False).apply(get_atm_iv).reset_index(drop=True)

# Drop the temporary 'diff' column
df.drop(columns=['diff'], inplace=True)


In [3]:
df=pd.read_csv('temp.csv')

In [4]:
df=df[['instrument','underlying','expiry','strike','put_call','multiplier',
    'quantity','time_to_expiry','rate','cost_of_carry_rate','vol','spot','atm_ivol']]

## Spot Vol Shocks

In [5]:
spot_vol_shocks = portrisk.crypto.CryptoSpotVolShocks(crypto_parameters)

In [6]:
post_stress, col = spot_vol_shocks.apply_spot_vol_shocks(df)

In [51]:
portrisk.core.black_scholes.bs_pricing(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')
portrisk.core.black_scholes.calc_vega(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')

# np.float64(3606.873391368419)
# np.float64(3700.518323796012)

np.float64(93.56524889234046)

In [8]:
post_stress

,instrument,underlying,expiry,strike,put_call,multiplier,quantity,time_to_expiry,rate,cost_of_carry_rate,vol,spot,atm_ivol,spot_shock,vol_shock,scenario_1,scenario_2
0,BTC-25OCT24-50000-P,BTC,2024-10-25,50000.0,put,1,-25.0,0.221918,0.03,default,0.5991,54770.44,0.5991,0.15,0,35040.355018,47574.312338
1,BTC-25OCT24-84000-C,BTC,2024-10-25,84000.0,call,1,0.0,0.221918,0.03,default,0.6461,54770.44,0.5991,0.15,0,0.000000,0.000000
2,BTC-27DEC24-105000-C,BTC,2024-12-27,105000.0,call,1,-8.8,0.394521,0.03,default,0.6975,54770.44,0.6305,0.15,0,-6262.417206,-10261.216646
3,BTC-27DEC24-110000-C,BTC,2024-12-27,110000.0,call,1,-28.9,0.394521,0.03,default,0.7063,54770.44,0.6305,0.15,0,-17850.476667,-29335.441013
4,BTC-27DEC24-115000-C,BTC,2024-12-27,115000.0,call,1,-1.7,0.394521,0.03,default,0.7168,54770.44,0.6305,0.15,0,-923.461242,-1520.939717
5,BTC-27DEC24-120000-C,BTC,2024-12-27,120000.0,call,1,95.0,0.394521,0.03,default,0.7275,54770.44,0.6305,0.15,0,45744.315466,75477.005640
6,BTC-27DEC24-43000-P,BTC,2024-12-27,43000.0,put,1,31.4,0.394521,0.03,default,0.6483,54770.44,0.6305,0.15,0,-29699.426796,-41080.407154
7,BTC-27DEC24-44000-P,BTC,2024-12-27,44000.0,put,1,20.6,0.394521,0.03,default,0.6452,54770.44,0.6305,0.15,0,-21082.226067,-29211.916499
8,BTC-27DEC24-45000-P,BTC,2024-12-27,45000.0,put,1,14.0,0.394521,0.03,default,0.6424,54770.44,0.6305,0.15,0,-15454.889557,-21452.340728
9,BTC-27DEC24-47000-P,BTC,2024-12-27,47000.0,put,1,-9.0,0.394521,0.03,default,0.6380,54770.44,0.6305,0.15,0,11458.331221,15962.359259


## Vol Surface Shocks

In [9]:
vol_surface_shocks = portrisk.crypto.CryptoVolSurfaceShocks(vol_surface_shocks_parameters)

In [10]:
vol_surface_shocks.run(df)

[{'product': 'BTC',
  'measure': 'Concentration',
  'value': np.float64(-69310.02641558654),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'TermStructure',
  'value': np.float64(-149224.79546228904),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'Skew',
  'value': np.float64(-15.517868167288324),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'BidAsk',
  'value': np.float64(-5557.740533208233),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'BTC',
  'measure': 'Sum',
  'value': np.float64(-224108.0802792511),
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'Sum',
  'measure': 'Concentration',
  'value': -69310.02641558654,
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'product': 'Sum',
  'measure': 'TermStructure',
  'value': -149224.79546228904,
  'group': None,
  'liquidity': False,
  'type': 'ix'},
 {'p

## Ivol Timing

In [2]:
df = pd.read_excel('ivol_timing.xlsx')
df['time'] = pd.to_datetime(df['timestamp'], unit='ms')

In [28]:
# Volatility Index

from datetime import datetime, timedelta
import requests

ccy='BTC'
end_timestamp = int(datetime.now().timestamp() * 1000)
time_24_hours_ago = datetime.now() - timedelta(hours=24)
start_timestamp = int(time_24_hours_ago.timestamp() * 1000)
url = f"https://test.deribit.com/api/v2/public/get_volatility_index_data"

r = requests.get(url, params = {'currency': ccy,
                                'end_timestamp': end_timestamp,
                                'start_timestamp': start_timestamp,
                                'resolution':3600})

df_ivol_index = pd.DataFrame(r.json()['result']['data']).rename(columns={0:'timestamp',1:'open',2:'high',3:'low',4:'close'})
df_ivol_index['time'] = pd.to_datetime(df_ivol_index['timestamp'], unit='ms')
df_ivol_index['ivol_index'] = df_ivol_index['close']


In [29]:
import plotly.graph_objects as go

# Create separate traces for buy and sell
buy_trace = go.Scatter(x=df[df['direction'] == ' buy']['time'], y=df[df['direction'] == ' buy']['iv'],
                       mode='markers', marker=dict(color='blue', size=5), name='Buy')
sell_trace = go.Scatter(x=df[df['direction'] == ' sell']['time'], y=df[df['direction'] == ' sell']['iv'],
                        mode='markers', marker=dict(color='red', size=5), name='Sell')
# Line trace for ivol data
line_trace = go.Scatter(
    x=df_ivol_index['time'],
    y=df_ivol_index['ivol_index'],
    mode='lines',
    line=dict(color='green'),
    name='Ivol Index'
)

# Create figure with both traces
fig = go.Figure([buy_trace, sell_trace,line_trace])

# Update layout
fig.update_layout(title='IV vs Time',
                  xaxis_title='Time',
                  yaxis_title='IV')

# Show plot
fig.show()